In [9]:
import os
import atexit
import sys
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
import findspark
from sparkhpc import sparkjob
import pandas
from pyspark.sql import SparkSession

#Exit handler to clean up the Spark cluster if the script exits or crashes
def exitHandler(sj,sc):
    try:
        print('Trapped Exit cleaning up Spark Context')
        sc.stop()
    except:
        pass
    try:
        print('Trapped Exit cleaning up Spark Job')
        sj.stop()
    except:
        pass

findspark.init()

#Parameters for the Spark cluster
nodes=3
tasks_per_node=8 
memory_per_task=1024 #1 gig per process, adjust accordingly
# Please estimate walltime carefully to keep unused Spark clusters from sitting 
# idle so that others may use the resources.
walltime="2:00" #60 min 
os.environ['SBATCH_PARTITION']='breezy' #Set the appropriate ARC partition

sj = sparkjob.sparkjob(
     ncores=nodes*tasks_per_node,
     cores_per_executor=tasks_per_node,
     memory_per_core=memory_per_task,
     walltime=walltime
    )

try:
    print('Cleaning up Spark Job')
    sj.stop()
except:
    pass

sj.wait_to_start()

try:
    print('Cleaning up Spark Context')
    sc.stop()
except:
    pass

sc = sj.start_spark()

#Register the exit handler                                                                                                     
atexit.register(exitHandler,sj,sc)

#You need this line if you want to use SparkSQL
sqlCtx=SQLContext(sc)


INFO:sparkhpc.sparkjob:Submitted batch job 630701

INFO:sparkhpc.sparkjob:Submitted cluster 0


Cleaning up Spark Job
Cleaning up Spark Context


In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dfTest').getOrCreate()
df = spark.read.csv('../Dataset/flight_perf_2013_1.csv',inferSchema=True,header=True)

### First create a view to use as a table

In [11]:
df.createOrReplaceTempView("flights")
#The first way is using the functions we used in dataframes
sqlCtx.table("flights").select("Origin","Dest","Distance").orderBy("Distance", ascending=False).show()

+------+----+--------+
|Origin|Dest|Distance|
+------+----+--------+
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
+------+----+--------+
only showing top 20 rows



In [7]:
#But you canalso use native SQLs
sqlCtx.sql("select Origin, Dest, Distance from flights order by Distance desc").show()

+------+----+--------+
|Origin|Dest|Distance|
+------+----+--------+
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
|   HNL| JFK|  4983.0|
+------+----+--------+
only showing top 20 rows



## Find max total delay for each AirlineID (totalDelay = ArrDelay-DepDelay)

In [8]:
sqlCtx.sql("select AirlineID, max(ArrDelay-DepDelay) as MaxTotalDelay \
            from flights group by AirlineID").show(50) # It is possible to use backslash to write multiline queries

+---------+-------------+
|AirlineID|MaxTotalDelay|
+---------+-------------+
|    20398|        149.0|
|    19690|         59.0|
|    20409|        154.0|
|    20378|        182.0|
|    19790|        212.0|
|    20304|        161.0|
|    20436|        102.0|
|    19393|        161.0|
|    20355|        150.0|
|    19805|        139.0|
|    19930|        112.0|
|    20366|        142.0|
|    20363|        127.0|
|    19977|        146.0|
|    21171|        108.0|
|    20437|        122.0|
+---------+-------------+



## Find airlineIDs with negative arrival delay count > 30000

In [9]:
#When it is asked to do some operations for 'each' specific feature value,
#Then you need group by statement.
#If the criteria is based  on a future for a resulting group, then you need to use 'having'
#Otherwise, 'where' statement is used for filtering the data
sqlCtx.sql("select AirlineID, count(*) from flights \
            where ArrDelay < 0 \
            group by AirlineID having count(*) > 30000").show()

+---------+--------+
|AirlineID|count(1)|
+---------+--------+
|    19790|   39867|
|    19393|   55018|
|    20366|   30895|
+---------+--------+



## Find all the origins with their cancelled flight counts

In [10]:
sqlCtx.sql("select Origin, count(*) from flights where Cancelled > 0 group by Origin").show(2000)

+------+--------+
|Origin|count(1)|
+------+--------+
|   BGM|       1|
|   INL|       3|
|   MSY|      27|
|   GEG|      11|
|   SNA|      25|
|   BUR|      37|
|   GRB|      36|
|   GTF|       3|
|   IDA|       8|
|   GRR|      22|
|   JLN|       2|
|   EUG|      16|
|   GSO|      25|
|   MYR|       4|
|   PVD|       5|
|   OAK|      36|
|   FAR|      20|
|   FSM|       2|
|   MSN|      24|
|   MQT|       2|
|   COD|       4|
|   BTM|       7|
|   SCC|       2|
|   ESC|       6|
|   DCA|     122|
|   MLU|      17|
|   CID|      52|
|   GTR|       2|
|   LWS|       2|
|   WRG|       1|
|   HLN|       2|
|   CIU|       1|
|   LEX|      19|
|   RDM|       7|
|   ORF|      27|
|   SCE|       5|
|   EVV|       6|
|   KTN|       6|
|   CRW|      11|
|   CWA|      20|
|   SAV|       6|
|   GCK|       2|
|   TRI|       4|
|   CDV|       2|
|   CMH|      27|
|   MOD|       9|
|   TYR|       6|
|   ADK|       2|
|   CAK|       5|
|   CHO|       6|
|   MOB|      10|
|   PNS|      14|
|   CEC|  

## Show the top 10 of previous list in descending order

In [12]:
#the first part is the same, we need to add order by statement and limit the result to 10
sqlCtx.sql("select Origin, count(*) from flights \
            where Cancelled > 0 group by Origin \
            order by count(*) desc limit 10").show(50)

+------+--------+
|Origin|count(1)|
+------+--------+
|   ORD|     604|
|   DFW|     512|
|   ATL|     301|
|   IAH|     240|
|   EWR|     240|
|   DTW|     236|
|   MSP|     211|
|   SLC|     204|
|   LGA|     185|
|   SFO|     169|
+------+--------+



## Show the minimum flight distance for origins which are in the top 10 of cancelled flight count

In [12]:
sqlCtx.sql("select Origin, min(Distance) from flights \
            where Origin IN \
               (select Origin from flights where Cancelled > 0 \
                group by Origin order by count(*) desc limit 10) \
            group by Origin").show()

+------+-------------+
|Origin|min(Distance)|
+------+-------------+
|   IAH|         74.0|
|   LGA|         96.0|
|   EWR|         80.0|
|   DFW|         89.0|
|   SFO|         77.0|
|   ATL|         83.0|
|   ORD|         67.0|
|   DTW|         56.0|
|   MSP|         76.0|
|   SLC|        150.0|
+------+-------------+



## Find sum(abs(DepDelay) + abs(ArrDelay)) in 24th day of the month
(Sum of total delays for 24th day of the month. We use abs function to penalize early arrivals/departures too)

In [13]:
sqlCtx.sql("select sum(abs(DepDelay) + abs(ArrDelay)) from flights where DayOfMonth=24").show()

+------------------------------------+
|sum((abs(DepDelay) + abs(ArrDelay)))|
+------------------------------------+
|                            642433.0|
+------------------------------------+



## For each day of month find the number of cancelled flights, and return:

## LOW if count < 100
## NORMAL if count < 400
## CRITICAL otherwise

In [18]:
sqlCtx.sql("select DayOfMonth, count(*), \
                case when count(*) < 100 then 'LOW' \
                when count(*) < 400 then 'NORMAL' \
                else 'CRITICAL' end as LEVEL \
            from flights where cancelled=1 group by DayofMonth").show(35)
#For the 3rd, column, the first is:
#CASE [conditions] END

+----------+--------+--------+
|DayOfMonth|count(1)|   LEVEL|
+----------+--------+--------+
|        31|     373|  NORMAL|
|        28|     556|CRITICAL|
|        26|     173|  NORMAL|
|        27|     592|CRITICAL|
|        12|     150|  NORMAL|
|        22|     138|  NORMAL|
|         1|     116|  NORMAL|
|        13|     280|  NORMAL|
|        16|     339|  NORMAL|
|         6|      80|     LOW|
|         3|     123|  NORMAL|
|        20|     226|  NORMAL|
|         5|      52|     LOW|
|        19|      52|     LOW|
|        15|     555|CRITICAL|
|         9|     141|  NORMAL|
|        17|     242|  NORMAL|
|         4|     136|  NORMAL|
|         8|      78|     LOW|
|        23|     201|  NORMAL|
|         7|      87|     LOW|
|        10|     118|  NORMAL|
|        25|     353|  NORMAL|
|        24|     335|  NORMAL|
|        29|     504|CRITICAL|
|        21|     162|  NORMAL|
|        11|     279|  NORMAL|
|        14|     189|  NORMAL|
|         2|     145|  NORMAL|
|       

## Airline with the maximum  avg flight distance between month's 10th and 20th days inclusive
There are 2 ways to do it:
1) Find the max value, run an outer query to find the airlines having value equal to the one found in the inner one
2) Sort in descending order with avg flight distance, take the first one

In [22]:
sqlCtx.sql("select AirlineID from flights \
            where DayOfMonth >= 10 and DayofMonth <= 20 \
            group by AirlineID having AVG(Distance) = \
                (select max(avgDistance) from \
                    (select avg(Distance) as avgDistance from flights \
                    where DayOfMonth >= 10 and DayofMonth <= 20 \
                    group by AirlineID)\
                )").show()
#The second way:
sqlCtx.sql("select AirlineID from flights \
            where DayOfMonth >= 10 and DayofMonth <= 20 \
            group by AirlineID \
            order by avg(Distance) desc limit 1").show()

+---------+
|AirlineID|
+---------+
|    21171|
+---------+

+---------+
|AirlineID|
+---------+
|    21171|
+---------+



## Find the origin that has the most flights to destination 'ATL' with distance > 600

In [25]:
#Exercise: try using the first approach in the previous question
sqlCtx.sql("select Origin, count(*) from flights \
            where Distance > 600 AND Dest = 'ATL' \
            group by Origin order by count(*) DESC limit 1").show()

+------+--------+
|Origin|count(1)|
+------+--------+
|   LGA|     878|
+------+--------+



## Find the state with the maximum distinct number of origins

In [20]:
sqlCtx.sql("select OriginState, count(distinct Origin) as dOrig \
            from flights group by OriginState order by dOrig desc limit 1").show()

+-----------+-----+
|OriginState|dOrig|
+-----------+-----+
|         CA|   26|
+-----------+-----+



## Find the number of originState/destState pair with number of flights and avg abs total delays

In [27]:
#When you want to group using multiple columns, you need paranthesis
sqlCtx.sql("select (OriginState, DestState) as StatePair, \
                count(*), avg(abs(DepDelay)+abs(ArrDelay)) as avgDelay \
            from flights group by (OriginState, DestState) order by count(*) desc").show(200)

+---------+--------+------------------+
|StatePair|count(1)|          avgDelay|
+---------+--------+------------------+
| [CA, CA]|   23665|24.234594501422045|
| [TX, TX]|   19441|29.711495830254407|
| [HI, HI]|    6153|22.228585575177423|
| [GA, FL]|    5373| 23.64552936775159|
| [FL, GA]|    5373|25.302638966872543|
| [AZ, CA]|    5060|21.339023421992856|
| [CA, AZ]|    5041| 21.04949111953702|
| [FL, NY]|    4680| 35.53791315618924|
| [NY, FL]|    4674|33.441875540190146|
| [CA, TX]|    4547|27.085021144001782|
| [TX, CA]|    4534| 32.11128456735058|
| [CA, NV]|    4325| 19.45985571328834|
| [NV, CA]|    4302|24.096932802622337|
| [FL, TX]|    3409| 32.29871680095494|
| [TX, FL]|    3407|29.828426999702646|
| [CO, CA]|    3237| 32.05505443234836|
| [CA, CO]|    3223| 26.26595744680851|
| [FL, FL]|    2893|25.934123387940048|
| [LA, TX]|    2706| 38.19619047619047|
| [TX, LA]|    2701|31.573720397249808|
| [WA, CA]|    2566| 23.79161770466118|
| [CA, WA]|    2562|25.266036993309722|


## Find destinations with at least one cancellation at every day of the month

In [35]:
#The problem in the tutorial was, I forgot one criteria in the inner loop, misleading me while preparing the question.
#There is no destination with at least 500 cancellations in each day of the month!
sqlCtx.sql("select distinct Dest from flights f \
            where (select count(*) from \
                    (select DayOfMonth from flights f2 \
                        where f2.Dest = f.Dest and cancelled>0 \
                        group by DayOfMonth having count(*) >= 1\
                    )\
                   ) = 31").show(200)
#In the outer loop, we pick one destination.
#In the inner loop, we find all cancelled flights going to that destination, group by dayofmonth and
#count the number of rows. It the result is 31, we were able to find at least one cancelled flight in
#each day of the month.

+----+
|Dest|
+----+
| IAH|
| DFW|
| SFO|
| ORD|
+----+



## For every airport in California, print :
    
## LOW TRAFFIC if flight count < 500
## DESCENT TRAFFIC if < 2000
## HIGH TRAFFIC otherwise

In [36]:
sqlCtx.sql("select Origin, count(*), \
                case when count(*) < '500' then 'LOW TRAFFIC' \
                when count(*) < 2000 then 'DESCENT TRAFFIC' \
                else 'HIGH TRAFFIC' end as TrafficLevel \
            from flights where OriginState = 'CA' \
            group By Origin").show(50)
#First we filter the originstate as CALIFORNIA, and group using the airport code.
#The 3rd column is constructed depending on the number of flights for each group(airport)

+------+--------+---------------+
|Origin|count(1)|   TrafficLevel|
+------+--------+---------------+
|   SNA|    3355|   HIGH TRAFFIC|
|   BUR|    2051|   HIGH TRAFFIC|
|   OAK|    3562|   HIGH TRAFFIC|
|   MOD|      92|    LOW TRAFFIC|
|   CEC|      85|    LOW TRAFFIC|
|   SJC|    3156|   HIGH TRAFFIC|
|   RDD|      96|    LOW TRAFFIC|
|   LGB|    1083|DESCENT TRAFFIC|
|   BFL|     307|    LOW TRAFFIC|
|   IPL|      62|    LOW TRAFFIC|
|   PSP|    1127|DESCENT TRAFFIC|
|   SBA|     917|DESCENT TRAFFIC|
|   IYK|      61|    LOW TRAFFIC|
|   CIC|      93|    LOW TRAFFIC|
|   ACV|     264|    LOW TRAFFIC|
|   MRY|     481|    LOW TRAFFIC|
|   SMF|    3435|   HIGH TRAFFIC|
|   MMH|      96|    LOW TRAFFIC|
|   SFO|   13218|   HIGH TRAFFIC|
|   FAT|     928|DESCENT TRAFFIC|
|   SMX|     107|    LOW TRAFFIC|
|   ONT|    1773|DESCENT TRAFFIC|
|   LAX|   17563|   HIGH TRAFFIC|
|   SAN|    6123|   HIGH TRAFFIC|
|   SBP|     400|    LOW TRAFFIC|
|   CLD|     229|    LOW TRAFFIC|
+------+------

## Find number of flights of the airports having flights to the airports with the least number of outgoing flights

In [39]:
#The first step is finding the airports with the least number of flights
#By using that list, you can filter the flights, going to those destinations
#The most inner loop finds the minimum number of flights from ANY airport, let call it X
#One outer loop finds the airports with X many flights, let call the list L
#Second outer loop finds the list of airports where the destination is in the list L. Lets call this list M
#The most outer loop finds the number of flights for the airports in list M
sqlCtx.sql("select Origin, count(*) from flights f1 \
            where f1.Origin IN \
                (select f2.Origin from flights f2 \
                where f2.Dest IN \
                    (select Origin from flights f3 \
                        group by Origin \
                        having count(*) = \
                            (select min(c) from \
                                (select count(*) as c from flights f4 \
                                group by Origin)\
                            )\
                    )\
                ) group by Origin").show()

+------+--------+
|Origin|count(1)|
+------+--------+
|   HNL|    4388|
|   DEN|   17850|
|   ANC|    1228|
+------+--------+

